## 1. Import necessary module

In [22]:
from scipy.optimize import milp, LinearConstraint
from qiskit_optimization import QuadraticProgram
from qiskit import QiskitError
import numpy as np
#from qiskit_optimization.algorithms import MinimumEigenOptimizer


## 2. Create a demo problem

### Example 1


$max \ x_1$

$-x_1+x_2 \leq 1 \\ 3x_1+2x_2 \leq 12 \\ 2x_1+3x_2 \leq 12$

$x_1, x_2 \in \mathcal{Z}$

In [4]:
c = -np.array([0, 1])
A = np.array([[-1, 1], [3, 2], [2, 3]])
b_u = np.array([1, 12, 12])
b_l = np.full_like(b_u, -np.inf)

constraints = LinearConstraint(A, b_l, b_u)
integrality = np.ones_like(c)
res = milp(c=c, constraints=constraints, integrality=integrality)

print('x1:', res.x[0], 'x2: ', res.x[1])


x1: 1.0 x2:  2.0


In [41]:
qp = QuadraticProgram('example-1')
qp.integer_var(name='x1')
qp.integer_var(name='x2')
qp.maximize(linear={'x2': 1})
#qp.minimize(linear={'x2': 1})
qp.linear_constraint({'x1': -1, 'x2': 1},'<=',1)
qp.linear_constraint({'x1': 3, 'x2': 2},'<=',12)
qp.linear_constraint({'x1': 2, 'x2': 3},'<=',12)

print(qp)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: example-1

Maximize
 obj: x2
Subject To
 c0: - x1 + x2 <= 1
 c1: 3 x1 + 2 x2 <= 12
 c2: 2 x1 + 3 x2 <= 12

Bounds

Generals
 x1 x2
End



## 3. Build a Converter

The converter should be able to extract necessary information from the `QuadraticProblem` class and input it to the milp solver.

In [31]:
## First, Check if qp is linear


## Check objective is linear
if qp.objective.quadratic.to_dict()!={}:
    raise QiskitError('Obejective function is not linear!')

## Check constrains are linear
if qp.quadratic_constraints:
    raise QiskitError('Constraints are not linear!')

## Sense: 1 for minimization and -1 for maximization 
sense = qp.objective.sense.value

## cost function for milp solver
c = qp.objective.linear.to_array() * sense



In [34]:
qp.objective.linear.to_array()

array([0., 1.])

In [60]:
qp.get_linear_constraint(0).linear.to_array()

array([-1.,  1.])

In [62]:
qp.get_linear_constraint(0).sense

<ConstraintSense.LE: 0>